## Собираем данные помесячно с 1 запуском перерасчета на месяц

In [2]:
%reload_ext autoreload
%autoreload 2

In [8]:
from bot.test import TestAlgorithm, TestAccAlgorithm
from app.cache import LocalCache
from bot.vis import TaskProgress
from app.config import RunConfig, AccConfig

test_configs = [
    {
        'acc': AccConfig.from_repr_string('t2_2 [0] |u0.2 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t2_3 [0] |u0.2 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        ]
    },
    # {
    #     'acc': AccConfig.from_repr_string('t2 [0] |u0.25 d0|'),
    #     'bots':  [
    #         RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
    #     ]
    # },
    {
        'acc': AccConfig.from_repr_string('t3_2 [0] |u0.3 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t3_3 [0] |u0.3 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        ]
    },
    # {
    #     'acc': AccConfig.from_repr_string('t3 [0] |u0.35 d0|'),
    #     'bots':  [
    #         RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
    #     ]
    # },
    # {
    #     'acc': AccConfig.from_repr_string('t5 [0] |u0.4 d0|'),
    #     'bots':  [
    #         RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
    #         # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
    #     ]
    # },
    {
        'acc': AccConfig.from_repr_string('t5_2 [0] |u0.5 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            # RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        ]
    },
    {
        'acc': AccConfig.from_repr_string('t5_3 [0] |u0.5 d0|'),
        'bots':  [
            RunConfig.from_repr_string('MVID+ 4/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('RNFT+ 2/fan7:0/2 x l1 x 0.6(+x0.2)¤'),
            RunConfig.from_repr_string('MTLR+ 2/fan7:0/2 x l1 x 0.6¤ |u0.01 d0.0|'),
        ]
    },
]

def run_test(config):
    out = {
        '_': config['acc'],
        'acc': 1
    }
    for bot_cfg in config['bots']:
        out[bot_cfg.ticker] = 1
    
    for m in range(1,13):
        date_ = f"2024-{m:02d}-01"
    
        bot_alg_list = []
        for bot_cfg in config['bots']:
            
            test_alg = TestAlgorithm(do_printing=False, config=bot_cfg)
            new_config = test_alg.make_best_config(
                test_date=date_,
                prev_days=30,
                original_config=bot_cfg,
                use_big_make_alg=True,
            )
            new_config2 = test_alg.make_best_config(
                test_date=date_,
                prev_days=30,
                original_config=new_config,
                use_big_make_alg=True,
            )
            
            bot_alg_list.append(TestAlgorithm(do_printing=False, config=new_config2, use_cache=True))

        test_acc_alg = TestAccAlgorithm(
            do_printing=False, 
            config=config['acc'],
            bot_alg_list=bot_alg_list,
        )
        
        a_res = test_acc_alg.test(
            last_test_date=date_,
            test_days_num=30,
        )
        
        out['acc'] *= (1 + a_res['acc']['profit'] / 100)
        for bot_res in a_res['bots']:
            out[bot_res['config'].ticker] *= (1 + bot_res['profit_p'] / 100)
            
    out['acc'] = round(out['acc'], 2)
    for bot_cfg in config['bots']:
        out[bot_cfg.ticker] = round(out[bot_cfg.ticker], 2)

    return out

unique_configs = test_configs

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))

for cfg in test_configs:
    res = run_test(cfg)
    if res:
        results.append(res)
    progress.update_progress()

print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

print(results)

Запуск в 04:14
Закончено в 07:39, количество 6, длительность 3:24:50                                               
cache 356522 vals, used 1094578 times
[{'_': t2_2 [0] |u0.2 d0.0| , 'acc': 1.02, 'MVID': 1.03, 'RNFT': 0.94}, {'_': t2_3 [0] |u0.2 d0.0| , 'acc': 1.02, 'MVID': 1.03, 'RNFT': 0.98, 'MTLR': 0.93}, {'_': t3_2 [0] |u0.3 d0.0| , 'acc': 1.01, 'MVID': 1.02, 'RNFT': 0.98}, {'_': t3_3 [0] |u0.3 d0.0| , 'acc': 1.04, 'MVID': 1.03, 'RNFT': 0.98, 'MTLR': 0.96}, {'_': t5_2 [0] |u0.5 d0.0| , 'acc': 1.06, 'MVID': 1.03, 'RNFT': 0.98}, {'_': t5_3 [0] |u0.5 d0.0| , 'acc': 1.06, 'MVID': 1.04, 'RNFT': 0.99, 'MTLR': 0.99}]
